## Agent-based modeling and simulation of transport flows in Amsterdam
### This notebook handles the preprocessing of GVB data to deduce a population that can be used in an agent-based simulation of public transport in Amsterdam.

#### Load all packages

In [1]:
import networkx as nx
import warnings
import datetime
import seaborn as sn
sn.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

#### Load data for origin and destination per hour

In [16]:
origin = pd.read_csv('../../Data/20190403/Datalab_Reis_Herkomst_Uur_20190403.csv', sep=';')
origin = origin.dropna()
destination = pd.read_csv('../../Data/20190403/Datalab_Reis_Bestemming_Uur_20190402.csv', sep=';')
destination = destination.dropna()

In [3]:
# List of all stations in the sub-network
stations = ['Amsteldijk','Amstelstation','Amstelveenseweg','Beethovenstraat','Bullewijk','Burg.de Vlugtlaan', 
    'Centraal Station','Cornelis Troostplein','Dam','De Boelelaan/VU','De Pijp','Europaplein',
    'Gaasperplas','Gein','Gerrit v.d. Veenstraat','Heemstedestraat','Henk Sneevlietweg','IJsbaanpad', 
    'Isolatorweg','Jan v.Galenstraat','Keizersgracht','Koningsplein','Kraaienneststation','Leidseplein',
    'Maasstraat','Marie Heinekenplein','Minervaplein','Muntplein','Museumplein','Nieuwezijds Kolk',
    'Nieuwmarkt','Noord','Noorderpark','Olympiaplein','Olympiaweg','Olympisch stadion','Overamstel', 
    'Postjesweg','Prinsengracht','Reigersbos','Rijksmuseum','Roelof Hartplein','Rokin','Scheldestraat',
    'Spaklerweg','Spui','Station Bijlmer ArenA','Station Diemen-Zuid','Station Duivendrecht', 'Stadionweg'
    'Station Ganzenhoef','Station Holendrecht','Station Lelylaan','Station RAI','Station Sloterdijk', 
    'Strandvliet','Van Baerlestraat','Van der Madeweg','Venserpolder','Verrijn Stuartweg','Victorieplein', 
    'Vijzelgracht','VU medisch centrum','Waalstraat','Waterlooplein','Weesperplein','Wibautstraat','Station Zuid']

#### Change dates to datetime objects and add weekday column

In [17]:
origin['Datum'] = origin['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
origin['Weekdag'] = origin['Datum'].apply(lambda x: x.isoweekday())

destination['Datum'] = destination['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
destination['Weekdag'] = destination['Datum'].apply(lambda x: x.isoweekday())

#### Remove data where the Noord/Zuid line did not run yet
The Noord/Zuid line was opened on the 21st of July, so all data before August is not used.

In [18]:
print(datetime.datetime(2018, 8, 2))

2018-08-02 00:00:00


In [24]:
origin = origin[(origin['Datum'] >= datetime.datetime(2018, 8, 6)) & 
                 (origin['Datum'] < datetime.datetime(2019, 3, 25))]
destination = destination[(destination['Datum'] >= datetime.datetime(2018, 8, 6)) &
                          (destination['Datum'] < datetime.datetime(2019, 3, 25))]

#### Apply necessary filters for analysis
The first analysis is only applied on one type of day (1 = Mondays) and for the stations that are included in the sub-network.

In [26]:
o_maandag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 1)]
d_maandag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 1)]

In [27]:
o_dinsdag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 2)]
d_dinsdag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 2)]

In [28]:
o_woensdag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 3)]
d_woensdag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 3)]

In [29]:
o_donderdag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 4)]
d_donderdag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 4)]

In [30]:
o_vrijdag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 5)]
d_vrijdag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 5)]

In [31]:
o_zaterdag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 6)]
d_zaterdag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 6)]

In [32]:
o_zondag = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 7)]
d_zondag = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 7)]

#### Remove 'HalteCodes' that are not used in the sub-network

In [34]:
stopdict = {'ITW': 2, 'GN': 2, 'RAI': 2, 'VSW': 1, 'ND': 1, 'RKN': 1, 
            'WPP': 3, 'RGB': 2, 'HVW': 2, 'OAS': 2, 'CLL': 2, 'PJW': 2, 
            'BLW': 2, 'WBS': 3, 'HLD': 2, 'MDW': 0.6666666666666666, 'SVT': 2, 'DMZ': 1, 
            'BMR': 2, 'EPP': 1, 'WTC': 0.6666666666666666, 'DPP': 1, 'JLS': 2, 'DVD': 2, 
            'GZH': 1, 'NDP': 1, 'NMT': 3, 'VLN': 2, 'ASA': 3, 'KEN': 1, 
            'VPD': 1, 'SLW': 3, 'WLP': 3, 'HDS': 2, 'GPP': 1, 'VZG': 1, 
            'CS': 3, 'ASW': 2, 'STD': 2, '08070': 1, '09122': 1, 
            '09121': 1, '07094': 1, '07091': 1, '08071': 1, '09013': 1, 
            '07004': 1, '08076': 1, '08078': 1, '07307': 1, '07308': 1, 
            '07120': 2, '07108': 2, '09083': 2, '09082': 2, '07202': 2, 
            '07121': 2, '09078': 2, '09079': 2, '07084': 2, '07083': 2, 
            '07207': 2, '07205': 2, '07017': 2, '07303': 2, '07350': 2, 
            '09074': 2, '09075': 2, '07098': 2, '07097': 2, '07493': 2, 
            '07490': 2, '09119': 2, '09120': 2, '05036': 3, '05035': 3, 
            '06072': 3, '06071': 3, '06075': 3, '06076': 3, '06073': 3, 
            '06074': 3, '07015': 3, '09049': 3, '09051': 3, '05062': 3, 
            '05061': 3, '07006': 3, '07007': 3, '07322': 3, '07321': 3, 
            '07323': 3, '07324': 3, '06067': 3, '05063': 3, '05032': 3, 
            '05031': 3, '07016': 0.6666666666666666, '09114': 0.6666666666666666, 
            '09111': 0.6666666666666666, '09139': 4, '09140': 4, '05022': 4, 
            '05011': 5, '06089': 5, '05065': 5, '05070': 5, '05069': 5}

In [41]:
o_maandag = o_maandag[o_maandag['VertrekHalteCode'].isin(stopdict.keys())]
o_maandag['AantalLijnen'] = o_maandag['VertrekHalteCode'].map(stopdict)
o_maandag['AantalReizenNorm'] = o_maandag['AantalReizen']/o_maandag['AantalLijnen']

o_dinsdag = o_dinsdag[o_dinsdag['VertrekHalteCode'].isin(stopdict.keys())]
o_dinsdag['AantalLijnen'] = o_dinsdag['VertrekHalteCode'].map(stopdict)
o_dinsdag['AantalReizenNorm'] = o_dinsdag['AantalReizen']/o_dinsdag['AantalLijnen']

o_woensdag = o_woensdag[o_woensdag['VertrekHalteCode'].isin(stopdict.keys())]
o_woensdag['AantalLijnen'] = o_woensdag['VertrekHalteCode'].map(stopdict)
o_woensdag['AantalReizenNorm'] = o_woensdag['AantalReizen']/o_woensdag['AantalLijnen']

o_donderdag = o_donderdag[o_donderdag['VertrekHalteCode'].isin(stopdict.keys())]
o_donderdag['AantalLijnen'] = o_donderdag['VertrekHalteCode'].map(stopdict)
o_donderdag['AantalReizenNorm'] = o_donderdag['AantalReizen']/o_donderdag['AantalLijnen']

o_vrijdag = o_vrijdag[o_vrijdag['VertrekHalteCode'].isin(stopdict.keys())]
o_vrijdag['AantalLijnen'] = o_vrijdag['VertrekHalteCode'].map(stopdict)
o_vrijdag['AantalReizenNorm'] = o_vrijdag['AantalReizen']/o_vrijdag['AantalLijnen']

o_zaterdag = o_zaterdag[o_zaterdag['VertrekHalteCode'].isin(stopdict.keys())]
o_zaterdag['AantalLijnen'] = o_zaterdag['VertrekHalteCode'].map(stopdict)
o_zaterdag['AantalReizenNorm'] = o_zaterdag['AantalReizen']/o_zaterdag['AantalLijnen']

o_zondag = o_zondag[o_zondag['VertrekHalteCode'].isin(stopdict.keys())]
o_zondag['AantalLijnen'] = o_zondag['VertrekHalteCode'].map(stopdict)
o_zondag['AantalReizenNorm'] = o_zondag['AantalReizen']/o_zondag['AantalLijnen']

In [42]:
d_maandag = d_maandag[d_maandag['AankomstHalteCode'].isin(stopdict.keys())]
d_maandag['AantalLijnen'] = d_maandag['AankomstHalteCode'].map(stopdict)
d_maandag['AantalReizenNorm'] = d_maandag['AantalReizen']/d_maandag['AantalLijnen']

d_dinsdag = d_dinsdag[d_dinsdag['AankomstHalteCode'].isin(stopdict.keys())]
d_dinsdag['AantalLijnen'] = d_dinsdag['AankomstHalteCode'].map(stopdict)
d_dinsdag['AantalReizenNorm'] = d_dinsdag['AantalReizen']/d_dinsdag['AantalLijnen']

d_woensdag = d_woensdag[d_woensdag['AankomstHalteCode'].isin(stopdict.keys())]
d_woensdag['AantalLijnen'] = d_woensdag['AankomstHalteCode'].map(stopdict)
d_woensdag['AantalReizenNorm'] = d_woensdag['AantalReizen']/d_woensdag['AantalLijnen']

d_donderdag = d_donderdag[d_donderdag['AankomstHalteCode'].isin(stopdict.keys())]
d_donderdag['AantalLijnen'] = d_donderdag['AankomstHalteCode'].map(stopdict)
d_donderdag['AantalReizenNorm'] = d_donderdag['AantalReizen']/d_donderdag['AantalLijnen']

d_vrijdag = d_vrijdag[d_vrijdag['AankomstHalteCode'].isin(stopdict.keys())]
d_vrijdag['AantalLijnen'] = d_vrijdag['AankomstHalteCode'].map(stopdict)
d_vrijdag['AantalReizenNorm'] = d_vrijdag['AantalReizen']/d_vrijdag['AantalLijnen']

d_zaterdag = d_zaterdag[d_zaterdag['AankomstHalteCode'].isin(stopdict.keys())]
d_zaterdag['AantalLijnen'] = d_zaterdag['AankomstHalteCode'].map(stopdict)
d_zaterdag['AantalReizenNorm'] = d_zaterdag['AantalReizen']/d_zaterdag['AantalLijnen']

d_zondag = d_zondag[d_zondag['AankomstHalteCode'].isin(stopdict.keys())]
d_zondag['AantalLijnen'] = d_zondag['AankomstHalteCode'].map(stopdict)
d_zondag['AantalReizenNorm'] = d_zondag['AantalReizen']/d_zondag['AantalLijnen']

#### Analyze patterns by day

In [45]:
d_zaterdag.head()

,Datum,UurgroepOmschrijving (van aankomst),AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalReizen,Weekdag,AantalLijnen,AantalReizenNorm
1780474,2018-08-11 12:00:00,00:00 - 00:59,05011,Centraal Station,4.899218,52.378108,231,6,5.0,46.200000
1780475,2018-08-11 12:00:00,00:00 - 00:59,05022,Centraal Station,4.901043,52.377720,109,6,4.0,27.250000
1780476,2018-08-11 12:00:00,00:00 - 00:59,05032,Dam,4.895494,52.374929,26,6,3.0,8.666667
1780479,2018-08-11 12:00:00,00:00 - 00:59,05063,Dam,4.890646,52.372446,27,6,3.0,9.000000
1780480,2018-08-11 12:00:00,00:00 - 00:59,05069,Nieuwezijds Kolk,4.893731,52.376288,48,6,5.0,9.600000


#### Group by hour for further analysis

In [48]:
#  o_woensdag, o_donderdag, o_vrijdag, o_zaterdag, o_zondag
# d_woensdag, d_donderdag, d_vrijdag, d_zaterdag, d_zondag
days = len(set(o_maandag['Datum'].tolist()))
days

33

In [66]:
origingrouped = o_zaterdag[['UurgroepOmschrijving (van vertrek)','VertrekHalteNaam', 'AantalReizenNorm']]
origingrouped = origingrouped.groupby(['UurgroepOmschrijving (van vertrek)', 
                                'VertrekHalteNaam']).aggregate(['min', np.median, np.sum, max])

origingrouped['AantalReizenNorm','mean'] = origingrouped['AantalReizenNorm']['sum'].apply(lambda x: float(x/days))
origingrouped = origingrouped.reset_index()
origingrouped.columns = ['hour', 'stop', 'min', 'median', 'sum', 'max', 'mean']

In [67]:
destinationgrouped = d_zaterdag[['UurgroepOmschrijving (van aankomst)','AankomstHalteNaam', 'AantalReizenNorm']]
destinationgrouped = destinationgrouped.groupby(['UurgroepOmschrijving (van aankomst)', 
                                'AankomstHalteNaam']).aggregate(['min', np.median, np.sum, 'max'])

destinationgrouped['AantalReizenNorm','mean'] = destinationgrouped['AantalReizenNorm']['sum'].apply(lambda x: float(x/days))
destinationgrouped = destinationgrouped.reset_index()
destinationgrouped.columns = ['hour', 'stop', 'min', 'median', 'sum', 'max', 'mean']

In [65]:
origingrouped.to_csv('../Data/originzat.csv', index=False)
destinationgrouped.to_csv('../Data/destinationvrijdag.csv', index=False)

In [46]:
def gvbdict(origin, destination):
    gvborigin = {}
    gvbdestination = {}

    hours = list(set(origin['hour'].tolist()))

    for hour in hours:
        df = origin[origin['hour'] == hour]
        stops = df['stop'].tolist()
        means = df['mean'].tolist()
        total = df['mean'].sum()

        percentages = []
        for i in range(len(stops)):
            percentages.append(means[i]/total)

        gvborigin[hour[:2]] = (total, stops, percentages)

    for hour in hours:
        df = destination[destination['hour'] == hour]
        stops = df['stop'].tolist()
        means = df['mean'].tolist()
        total = df['mean'].sum()

        percentages = []
        for i in range(len(stops)):
            percentages.append(means[i]/total)

        gvbdestination[hour[:2]] = (total, stops, percentages)
        
    return gvborigin, gvbdestination

#### Create plot of distribution for a certain hour

In [ ]:
def plotgroup(hour, dataframe):
    dataframe = dataframe[dataframe['hour'] == hour]
    total = dataframe['mean'].sum()
    dataframe['percentage'] = dataframe['mean'].apply(lambda x: (x / total) * 100)
    dataframe = (dataframe.sort_values(by = 'percentage', ascending = False))
    dataframe = (dataframe.sort_values(by = 'stop', ascending = False))
    return dataframe.plot(kind = 'barh', x = 'stop', y = 'percentage', figsize = [10,10])

#### Save distribution for simulation